**Общая информация**

**Срок сдачи:** 21 октября 2019, 08:30 

**Штраф за опоздание:** по 0.5 балла за 24 часа задержки. Через 10 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0919, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Невыполнение PEP8 -0.5 баллов
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata, fetch_20newsgroups

from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KDTree
import scipy
from sklearn.metrics import pairwise_distances

%load_ext pycodestyle_magic

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

In [2]:
class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):
    def __init__(self, n_neighbors,
                 algorithm='brute', metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.algorithm = algorithm
        self.metric = metric

    def fit(self, X, y):
        self.X = X
        self.y = y
        self.classes = np.unique(y)
        self.n_classes = self.classes.shape[0]
        if self.algorithm == 'kd_tree':
            if self.metric != 'cosine':
                self.tree = KDTree(self.X, leaf_size=30, metric=self.metric)
            else:
                self.tree = KDTree(self.X, leaf_size=30)

    def predict(self, X):
        if self.algorithm == 'brute':
            dist = pairwise_distances(X, self.X, metric=self.metric)
            indeces = np.argpartition(dist, self.n_neighbors, axis=1)
            indeces = indeces[:, :self.n_neighbors]
        if self.algorithm == 'kd_tree':
            indeces = self.tree.query(X, k=self.n_neighbors)[1]
        classes = np.take(self.y, indeces)
        out = scipy.stats.mode(classes, axis=1)[0].T
        return np.squeeze(out)

    def predict_proba(self, X):
        if self.algorithm == 'brute':
            dist = scipy.spatial.distance.cdist(X, self.X, metric=self.metric)
            indeces = np.argpartition(dist,
                                      self.n_neighbors, axis=1)
            indeces = indeces[:, :self.n_neighbors]
        if self.algorithm == 'kd_tree':
            dist, indeces = self.tree.query(X, k=self.n_neighbors)
        classes = np.take(self.y, indeces)
        proba = np.zeros((X.shape[0], self.n_classes), dtype=float)
        for i in range(X.shape[0]):
            class_ind, counts = np.unique(classes[i], return_counts=True)
            proba[i][class_ind] = counts
        return proba / self.n_neighbors

    def score(self, X, y):
        return np.mean(np.mean(self.predict(X) == y))

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [3]:
iris = datasets.load_iris()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [5]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [6]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [7]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy

In [8]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 1.16 ms, total: 1.16 ms
Wall time: 839 µs


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [9]:
%time my_clf.fit(X_train, y_train)

CPU times: user 77 µs, sys: 9 µs, total: 86 µs
Wall time: 92.5 µs


In [10]:
%time clf.predict(X_test)

CPU times: user 2.34 ms, sys: 0 ns, total: 2.34 ms
Wall time: 2.66 ms


array([0, 2, 2, 2, 2, 2, 0, 2, 1, 0, 1, 0, 1, 0, 1])

In [11]:
%time my_clf.predict(X_test)

CPU times: user 3.23 ms, sys: 0 ns, total: 3.23 ms
Wall time: 2.85 ms


array([0, 2, 2, 2, 2, 2, 0, 2, 1, 0, 1, 0, 1, 0, 1])

In [12]:
%time clf.predict_proba(X_test)

CPU times: user 0 ns, sys: 5.79 ms, total: 5.79 ms
Wall time: 5.83 ms


array([[1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0.5, 0.5],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ]])

In [13]:
%time my_clf.predict_proba(X_test)

CPU times: user 1.07 ms, sys: 0 ns, total: 1.07 ms
Wall time: 771 µs


array([[1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 0.5, 0.5],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ]])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

In [14]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [15]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [16]:
%time clf.fit(X_train, y_train)

CPU times: user 1.24 ms, sys: 149 µs, total: 1.39 ms
Wall time: 1.17 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [17]:
%time my_clf.fit(X_train, y_train)

CPU times: user 675 µs, sys: 0 ns, total: 675 µs
Wall time: 684 µs


In [18]:
%time clf.predict(X_test)

CPU times: user 0 ns, sys: 2.42 ms, total: 2.42 ms
Wall time: 2.43 ms


array([0, 2, 0, 2, 1, 1, 0, 1, 1, 0, 2, 2, 1, 0, 2])

In [19]:
%time my_clf.predict(X_test)

CPU times: user 0 ns, sys: 2.35 ms, total: 2.35 ms
Wall time: 1.74 ms


array([0, 2, 0, 2, 1, 1, 0, 1, 1, 0, 2, 2, 1, 0, 2])

In [20]:
%time clf.predict_proba(X_test)

CPU times: user 2.9 ms, sys: 0 ns, total: 2.9 ms
Wall time: 2.46 ms


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [21]:
%time my_clf.predict_proba(X_test)

CPU times: user 1.34 ms, sys: 168 µs, total: 1.5 ms
Wall time: 1.51 ms


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [22]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [23]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [24]:
data = newsgroups['data']
target = newsgroups['target']

Переведите во всех документах все буквы в нижний регистр и замените во всех документах символы, не
являющиеся буквами и цифрами, на пробелы. Далее разбейте текста по пробельным символам на токены(термы/слова). Удалите текста, содержащие только пробелы.

In [25]:
# data_tok should be a list of lists of tokens for each line in data.

outp = [''.join(c if c.isalpha() or c.isdigit() else ' '
                for c in first).lower().split() for first in data]
target = [ans for text, ans in zip(outp, target) if text != []]
data_tok = list(filter(None, outp))
len(data_tok), len(target)

(11004, 11004)

In [26]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower() , map(' '.join, data_tok))), "please make sure that you lowercase the data and drop spaced texts"

Преобразуйте датасет в разреженную матрицу scipy.sparse.csr_matrix, где значение x в позиции (i, j)
означает, что в документе i слово j встретилось x раз

In [27]:
indptr = [0]
indices = []
data = []
vocabulary = {}

for d in data_tok:
    for term in d:
        index = vocabulary.setdefault(term, len(vocabulary))
        indices.append(index)
        data.append(1)
    indptr.append(len(indices))

vectorized = scipy.sparse.csr_matrix((data, indices, indptr),
                                     dtype=np.uint16)
target = np.array(target)
vectorized.shape, target.shape

((11004, 98749), (11004,))

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [28]:
from sklearn.model_selection import StratifiedKFold
folds = 3
scores = []
partition = StratifiedKFold(n_splits=folds)
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

for train_index, test_index in partition.split(vectorized, target):
    X_train, y_train = vectorized[train_index], target[train_index]
    my_clf.fit(vectorized[train_index], target[train_index])
    X_test, y_test = vectorized[test_index], target[test_index]
    scores.append(my_clf.score(vectorized[test_index], target[test_index]))
scores

KeyboardInterrupt: 

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальный score в среднем на валидации на 3 фолдах).
Постройте график зависимости среднего score от количества соседей. Можно рассмотреть число соседей от 1 до 10.

In [ ]:
from tqdm import tqdm_notebook


def my_cross_validation(samples, target,
                        partition, ans, key, metr='euclidean'):
    ans[key] = []
    for current_n_neighb in tqdm_notebook(range(1, 11)):
        my_clf = MyKNeighborsClassifier(n_neighbors=current_n_neighb,
                                        algorithm='brute', metric=metr)
        scores = []
        for train_index, test_index in partition.split(samples, target):
            X_train, y_train = samples[train_index], target[train_index]
            my_clf.fit(X_train, y_train)
            X_test, y_test = samples[test_index], target[test_index]
            scores.append(my_clf.score(X_test, y_test))
        ans[key].append(np.mean(scores))
    return ans

In [ ]:
ans = my_cross_validation(vectorized, target, partition, {}, 'brute')
ans

In [ ]:
ans = my_cross_validation(vectorized, target,
                          partition, ans, 'brute_cosine', 'cosine')
ans

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(np.arange(1, 11), ans['brute'], label='brute euclidean')
plt.plot(np.arange(1, 11), ans['brute_cosine'], label='brute cosine')
plt.legend(['brute euclidean', 'brute cosine'],  loc='upper left')
plt.xlabel('K_Neigbors')
plt.ylabel('Score')
plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformed = TfidfTransformer(use_idf=True,
                               smooth_idf=True).fit_transform(vectorized)
transformed

In [ ]:
ans_tf_idf = my_cross_validation(transformed, target, partition, {}, 'brute')
ans_tf_idf

In [ ]:
ans_tf_idf = my_cross_validation(transformed, target,
                                 partition, ans_tf_idf,
                                 'brute_cosine', 'cosine')
ans_tf_idf

In [ ]:
plt.plot(np.arange(1, 11), ans_tf_idf['brute'],
         label='after TFiDF brute euclidean')
plt.plot(np.arange(1, 11), ans_tf_idf['brute_cosine'],
         label='ater TFiDF brute cosine')
plt.legend(['after TFiDF brute euclidean', 'ater TFiDF brute cosine'],
           loc='upper left')
plt.xlabel('K_Neigbors')
plt.ylabel('Score')
plt.show()

In [ ]:
# Убрали из рассмотрения число соседей равное единице

best_param_eucliden = np.argmax(ans['brute'][1:]) + 2
best_param_cosine = np.argmax(ans['brute_cosine'][1:]) + 2
best_param_tf_idf = np.argmax(ans_tf_idf['brute_cosine'][1:]) + 2
print('best param eculid: ',
      best_param_eucliden, ans['brute'][best_param_eucliden - 1])
print('best param cosine: ',
      best_param_cosine, ans['brute_cosine'][best_param_cosine - 1])
print('best param tf_idf: ',
      best_param_tf_idf, ans_tf_idf['brute_cosine'][best_param_tf_idf - 1])

Таким образом, лучший результат достингут после TfiDF пробазования на косинусной метрике при числе соседей - 9

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf преобразование( sklearn.feature_extraction.text.TfidfTransformer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])


In [ ]:
test_target = newsgroups['target']
test_data = newsgroups['data']

In [ ]:
outp = [''.join(c if c.isalpha() or c.isdigit() else ' '
                for c in first).lower().split() for first in test_data]
test_target = [ans for text, ans in zip(outp, test_target) if text != []]
test_data_tok = list(filter(None, outp))
len(test_data_tok), len(test_target)

In [ ]:
t_indptr = [0]
t_indices = []
t_data = []

train_vocabulary = vocabulary
for d in t_data_tok:
    for term in d:
        if term in train_vocabulary:
            index = train_vocabulary[term]
            t_indices.append(index)
            t_data.append(1)
        else:
            continue
    t_indptr.append(len(t_indices))

test_vectorized = scipy.sparse.csr_matrix((t_data, t_indices, t_indptr),
                                          dtype=np.uint16)
t_target = np.array(test_target)
test_vectorized

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
test_transformed = TfidfTransformer().fit_transform(test_vectorized)
test_transformed

Оценим качество(score) вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.

In [ ]:
transformed.shape, target.shape

In [ ]:
test_transformed.shape, test_target.shape

In [ ]:
my_best_clf = MyKNeighborsClassifier(n_neighbors=9,
                                     algorithm='brute', metric='cosine')
my_best_clf.fit(transformed, target)
my_best_clf.score(test_transformed, test_target)

Точность предсказания упала, так как словарь обучающей выборки не соответствует словарю тестовой